# Implementing N-grams
* **Name:** Mohammad Mahdi Salmani

In [1]:
import random
import re
import nltk
from nltk import bigrams
from nltk.tokenize import word_tokenize
from collections import Counter
from collections import defaultdict

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Q 3.1: Load and Preprocess Dataset

In [3]:
def load_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text[1:]

def normalize_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [4]:
# Load the text file
file_path = "./data/Tarzan.txt"
raw_text = load_text_file(file_path)
normalized_text = normalize_text(raw_text)

In [5]:
tokenized_text = word_tokenize(normalized_text)

print(f"Tokenized text: (len={len(tokenized_text)})")
print(tokenized_text)

Tokenized text: (len=72974)
['the', 'project', 'gutenberg', 'ebook', 'of', 'tarzan', 'lord', 'of', 'the', 'jungle', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorg', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'title', 'tarzan', 'lord', 'of', 'the', 'jungle', 'author', 'edgar', 'rice', 'burroughs', 'release', 'date', 'february', '12', '2024', 'ebook', '72938', 'language', 'english', 'original', 'publication', 

## Q 3.2: Implementing Bigram

In [ ]:
nltk.download('punkt')

In [242]:
def calculate_unigram_counts(tokenized_text):
    unigram_counts = Counter(tokenized_text)
    return unigram_counts

def calculate_bigram_counts(tokenized_text):
    bi_grams = list(bigrams(tokenized_text))
    bigram_counts = Counter(bi_grams)
    return bigram_counts

In [277]:
unigram_counts = calculate_unigram_counts(tokenized_text)
print('Unigrams:',unigram_counts)
bigram_counts = calculate_bigram_counts(tokenized_text)
print('Bigrams:',bigram_counts)

Unigrams: Counter({'the': 5548, 'of': 2562, 'and': 2444, 'to': 1618, 'he': 1509, 'a': 1341, 'his': 1071, 'that': 1068, 'in': 940, 'was': 845, 'had': 626, 'him': 613, 'it': 595, 'with': 588, 'for': 568, 'not': 558, 'i': 556, 'you': 510, 'as': 489, 'but': 462, 'they': 431, 'from': 413, 'blake': 411, 'be': 404, 'upon': 385, 'at': 383, 'were': 350, 'tarzan': 331, 'them': 304, 'is': 294, 'have': 291, 'by': 291, 'this': 265, 'their': 256, 'said': 253, 'there': 249, 'sir': 240, 'ibn': 238, 'thou': 235, 'would': 230, 'jad': 226, 'no': 222, 'one': 218, 'her': 213, 'great': 209, 'who': 209, 'or': 208, 'what': 207, 'we': 205, 'man': 203, 'an': 193, 'all': 182, 'stimbol': 181, 'she': 180, 'into': 175, 'when': 173, 'been': 170, 'are': 168, 'knights': 166, 'so': 156, 'me': 155, 'which': 154, 'could': 153, 'if': 150, 'will': 149, 'then': 147, 'before': 146, 'do': 144, 'nimmr': 143, 'where': 140, 'men': 137, 'did': 136, 'on': 135, 'my': 130, 'knight': 129, 'about': 129, 'two': 128, 'fahd': 123, 'throu

* Calculate bigram probablities

In [244]:
def calcBigramProb(unigramCounts, bigramCounts):
    bigram_probabilities = {}
    vocabulary_size = len(unigramCounts)
    for bigram, count in bigramCounts.items():
        word1, word2 = bigram[0], bigram[1]
        # # Apply add-one smoothing
        # smoothed_count = count + 1
        # # Add-one smoothing to unigram count as well
        # smoothed_unigram_count = unigramCounts[word1] + vocabulary_size
        if word1 not in bigram_probabilities:
            bigram_probabilities[word1] = {}
        bigram_probabilities[word1][word2] = count / unigramCounts[word1]
    return bigram_probabilities

In [246]:
tokens_len = len(tokenized_text)
unigram_probabilities = dict([(x[0],x[1]/tokens_len) for x in unigram_counts.items()])

bigram_probabilities = calcBigramProb(unigram_counts, bigram_counts)

In [293]:
def generate_next_token_bigram(token, bigram_probabilities):
    if token in bigram_probabilities:
        possible_next_tokens = list(bigram_probabilities[token].keys())
        weights = list(bigram_probabilities[token].values())
        return random.choices(possible_next_tokens, weights=weights)[0]
    else:
        # if bigram probablity is zero using unigram (to solve data sparsity problem)
        all_tokens = list(unigram_probabilities.keys())
        weights = list(unigram_probabilities.values())
        return random.choices(all_tokens, weights=weights)[0]

In [9]:
expressions = [
    "Knowing well the windings of the trail he",
    "For half a day he lolled on the huge back and"
]

In [295]:
# Complete expressions...
for i, expression in enumerate(expressions):
    print(f'[{i+1}] {expression}', end='  ')
    tokens = word_tokenize(normalize_text(expression))
    current_bigram = tokens[-1]
    # Generate 10 tokens
    for _ in range(10):
        next_token = generate_next_token_bigram(current_bigram, bigram_probabilities)
        print(next_token, end=' ')
        if next_token:
            tokens.append(next_token)
            current_bigram = next_token
        else:
            break
    print()

[1] Knowing well the windings of the trail he  aimed but as though he did not zeyd held at 
[2] For half a day he lolled on the huge back and  taken from pike and looked about the food and new 


## Q 3.4: Implementing Trigram:

In [138]:
from nltk.util import trigrams

def calculate_trigram_counts(tokenized_text):
    tri_grams = list(trigrams(tokenized_text))
    return Counter(tri_grams)

def calcTrigramProb(bigramCounts, trigramCounts):
    trigram_probabilities = {}
    for trigram, count in trigramCounts.items():
        word1, word2, word3 = trigram[0], trigram[1], trigram[2]
        if word1 not in trigram_probabilities:
            trigram_probabilities[word1] = {}
        if word2 not in trigram_probabilities[word1]:
            trigram_probabilities[word1][word2] = {}
        # # Apply add-one smoothing
        # smoothed_count = count + 1
        # smoothed_bigram_count = bigramCounts[word1][word2] + vocabulary_size
        trigram_probabilities[word1][word2][word3] = count / bigramCounts[word1][word2]
    return trigram_probabilities

def generate_next_token_trigram(trigram, trigram_probabilities):
    possible_next_tokens = [next_token for (first_token, second_token, next_token), prob in trigram_probabilities.items() if (first_token, second_token) == (trigram[-2], trigram[-1])]
    if possible_next_tokens:
        return random.choices(possible_next_tokens, weights=[trigram_probabilities[(trigram[-2], trigram[-1], next_token)] for next_token in possible_next_tokens])[0]
    else:
        return generate_next_token_bigram(trigram[-2:], bigram_probabilities)

In [139]:
# Train the trigram model
trigram_counts = calculate_trigram_counts(tokenized_text)
trigram_probabilities = calcTrigramProb(bigram_counts, trigram_counts)

In [140]:
# Complete expressions...
for i, expression in enumerate(expressions):
    tokens = word_tokenize(expression)
    current_trigram = (tokens[-3], tokens[-2], tokens[-1])
    # Generate 10 tokens
    for _ in range(10):
        next_token = generate_next_token_trigram(current_trigram, trigram_probabilities)
        if next_token:
            tokens.append(next_token)
            current_trigram = (current_trigram[-2], current_trigram[-1], next_token)
        else:
            break
    print(f'[{i+1}]', ' '.join(tokens))

[1] Knowing well the windings of the trail he took with seven great lions watching his slow way .
[2] For half a day he lolled on the huge back and join me ; but once within this tent it required


### Implementing N-gram:

In [32]:
from nltk.util import ngrams

class LanguageModel:
    def __init__(self, tokenized_text, n=5) -> None:
        self.n = n
        self.tokenized_text = tokenized_text

        unigram_counts = self.calculate_unigram_counts()
        self.unigram_probabilities = dict([(x[0],x[1]/len(tokenized_text)) for x in unigram_counts.items()])

    def calculate_unigram_counts(self):
        unigram_counts = Counter(self.tokenized_text)
        return unigram_counts

    def calculate_ngram_counts(self, n):
        n_grams = list(ngrams(self.tokenized_text, n))
        return Counter(n_grams)

    def calcNgramProb(self, n):
        n_gramCounts = self.calculate_ngram_counts(n)
        n_1_gramCounts = self.calculate_ngram_counts(n-1)

        ngram_probabilities = {}
        # vocabulary_size = len(n_1_gramCounts)
        for ngram, count in n_gramCounts.items():
            prefix = ngram[:-1]
            last_word = ngram[-1]
            # # Apply add-one smoothing
            # smoothed_count = count + 1
            # smoothed_prefix_count = ngramCounts.get(prefix, 0) + vocabulary_size
            if prefix not in ngram_probabilities:
                ngram_probabilities[prefix] = {}
            ngram_probabilities[prefix][last_word] = count / n_1_gramCounts[prefix]
        return ngram_probabilities

    def generate_next_token_ngram(self, ngram, ngram_probabilities):
        if ngram in ngram_probabilities:
            possible_next_tokens = list(ngram_probabilities[ngram].keys())
            weights = list(ngram_probabilities[ngram].values())
            return random.choices(possible_next_tokens, weights=weights)[0]
        else:
            if self.n > 3:
                n_2_gram_probs = self.calcNgramProb(self.n - 2)
                n_2_gram = tuple(ngram[2:])
                # print('\n\n*** ',n_2_gram,'\n\n') #TODO: delete this line
                return self.generate_next_token_ngram(n_2_gram, n_2_gram_probs)
            else:
                all_tokens = list(self.unigram_probabilities.keys())
                weights = list(self.unigram_probabilities.values())
                return random.choices(all_tokens, weights=weights)[0]

In [38]:
# Trigram model
trigram_model = LanguageModel(tokenized_text, n=3)
ngram_probabilities = trigram_model.calcNgramProb(n=3)
# Complete expressions...
print('Trigram model:')
for i, expression in enumerate(expressions):
    print(f'[{i+1}] {expression}', end='  ')
    tokens = word_tokenize(normalize_text(expression))
    current_ngram = tuple(tokens[-2:])
    # Generate 10 tokens
    for _ in range(10):
        next_token = trigram_model.generate_next_token_ngram(current_ngram, ngram_probabilities)
        print(next_token, end=' ')
        if next_token:
            tokens.append(next_token)
            current_ngram = tuple(tokens[-2:])
        else:
            break
    print()

Trigram model:
[1] Knowing well the windings of the trail he  came from the ground the brute came close guinalda closed 
[2] For half a day he lolled on the huge back and  overtake you without much loss of time how long he 


In [40]:
# Train the 5-gram model
_5_gram_model = LanguageModel(tokenized_text, n=5)
ngram_probabilities = _5_gram_model.calcNgramProb(n=5)

In [41]:
# Complete expressions...
print('5-gram model:')
for i, expression in enumerate(expressions):
    print(f'[{i+1}] {expression}', end='  ')
    tokens = word_tokenize(normalize_text(expression))
    current_ngram = tuple(tokens[-4:])
    # Generate 10 tokens
    for _ in range(10):
        next_token = _5_gram_model.generate_next_token_ngram(current_ngram, ngram_probabilities)
        print(next_token, end=' ')
        if next_token:
            tokens.append(next_token)
            current_ngram = tuple(tokens[-4:])
        else:
            break
    print()

5-gram model:
[1] Knowing well the windings of the trail he  took short cuts swinging through the branches of the trees 
[2] For half a day he lolled on the huge back and  join me but if she is with them youd better 
